In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import musical
import sys

# Ensure correct file paths 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from hrdtimer import utils as HRDTimerUtils

In [8]:
# DEFINE PATHS ----- need to modify based on where we will end up saving/providing the data.
PCAWG_timing_vcfs = "/Volumes/extSSD/park_lab/HRDTimer_Analysis/AA_NEW_TEST_RUN_PCAWG_Apr25_v2/Breast/timing"
breast_metadata_path = '/Users/michail/HMS Dropbox/Michail Andreopoulos/HRDTimer/data/metadata/pan_metadata_v3.csv'

In [12]:
breast_timing_samples = HRDTimerUtils.prepare_samples_for_timing(PCAWG_timing_vcfs)

breast_metadata_df = pd.read_csv(breast_metadata_path)
timing_sample_ids = list(breast_timing_samples.keys())

wgd_sample_ids = breast_metadata_df.query("sample in @timing_sample_ids and isWGD")["sample"].tolist()
hrd_wgd_sample_ids = breast_metadata_df.query("`HRDetect.isHRD` and organ == 'Breast' and isWGD")["sample"].tolist()

wgd_timing_samples = {s: breast_timing_samples[s] for s in wgd_sample_ids if s in breast_timing_samples}
hrd_wgd_timing_samples = {s: breast_timing_samples[s] for s in hrd_wgd_sample_ids if s in breast_timing_samples}

Processing Early samples:


Processing Files: 100%|██████████| 202/202 [00:22<00:00,  8.84it/s]


Processing Late samples:


Processing Files: 100%|██████████| 174/174 [00:20<00:00,  8.35it/s]


Processing NA samples:


Processing Files: 100%|██████████| 205/205 [00:20<00:00, 10.02it/s]


In [ ]:
def append_bootstrapped_probabilities(base_dir, mutation_types, thresh=0.001):
    catalog = musical.load_catalog('COSMIC_v3p2_SBS_WGS')
    catalog.restrict_catalog(tumor_type='Breast.AdenoCA', is_MMRD=False, is_PPD=False)
    W = catalog.W.reindex(mutation_types).fillna(0)

    sample_dirs = sorted(glob(os.path.join(base_dir, 'May13_Combined_DF_boostraps', 'bootstrap_*')))

    for boot_dir in tqdm(sample_dirs, desc="Processing Bootstraps"):
        sample_paths = [p for p in glob(os.path.join(boot_dir, '*.csv')) if not os.path.basename(p).startswith('Exposures_')]
        sample_ids = [os.path.splitext(os.path.basename(p))[0] for p in sample_paths]

        # Dict to store per-classification modified DataFrames
        all_sample_dfs = {sample: [] for sample in sample_ids}

        for classification in ['Early', 'Late', 'NA']:
            bootstrap_counts = []
            sample_dfs = {}

            for path, sample in zip(sample_paths, sample_ids):
                df = pd.read_csv(path)

                if classification == 'NA':
                    df_class = df[df['Classification'].isna() | (df['Classification'] == 'NA')].copy()
                else:
                    df_class = df[df['Classification'] == classification].copy()

                if df_class.empty:
                    continue

                df_class['MutationType'] = df_class['SBS96']
                counts = df_class['MutationType'].value_counts().reindex(mutation_types, fill_value=0)
                bootstrap_counts.append(counts)
                sample_dfs[sample] = df_class

            if not bootstrap_counts:
                continue

            H = pd.DataFrame(bootstrap_counts, index=sample_dfs.keys()).T
            H.index.name = 'MutationType'

            exposures, _ = musical.refit.refit(H, W, method='likelihood_bidirectional', thresh=thresh)

            for sample in sample_dfs:
                df = sample_dfs[sample]
                exp = exposures[sample]
                exp_norm = exp / exp.sum() if exp.sum() > 0 else pd.Series(0.0, index=exp.index)

                prob_matrix = W.mul(exp_norm, axis=1)
                prob_sum = prob_matrix.sum(axis=1)
                prob_df = prob_matrix.div(prob_sum, axis=0).fillna(0)
                prob_df = prob_df.rename(columns=lambda c: f'prob_{c}_boot').reset_index()

                df['Type'] = df['SBS96']

                df = df.merge(prob_df, left_on='Type', right_on='Type', how='left')

                all_sample_dfs[sample].append(df)

            # Optionally save exposures for each classification
            exposures.to_csv(os.path.join(boot_dir, f'Exposures_{classification}.csv'))

        # Write full merged DataFrames per sample
        for sample, dfs in all_sample_dfs.items():
            if dfs:  # only if there's content
                full_df = pd.concat(dfs, ignore_index=True)
                full_df.to_csv(os.path.join(boot_dir, f'{sample}.csv'), index=False)

# ---- Run Function ---- #
base_dir = '/Volumes/extSSD/park_lab/HRDTimer_Analysis/AAA_SIGNATURE_STABILITY_v2/PCAWG'
catalog = musical.load_catalog('COSMIC_v3p2_SBS_WGS')
mutation_types = catalog.W.index.tolist()

append_bootstrapped_probabilities(base_dir, mutation_types, thresh=0.001)

In [13]:
# Load COSMIC catalog
catalog = musical.load_catalog('COSMIC_v3p2_SBS_WGS')
catalog.restrict_catalog(tumor_type='Breast.AdenoCA', is_MMRD=False, is_PPD=False)
mutation_types = catalog.W.index.tolist()
W = catalog.W.reindex(mutation_types)

n_bootstraps = 200
# Save intermediate files in the temp dir so that they are not included in the commit
output_base_dir = '/Users/michail/HMS Dropbox/Michail Andreopoulos/HRDTimer/temp/PCAWG_Breast_boot_only_p_change'

# Ensure base output directory exists
os.makedirs(output_base_dir, exist_ok=True)

# Loop over bootstrap replicates
for i in tqdm(range(1, n_bootstraps + 1), desc="Bootstrapping"):
    # Create directory for this bootstrap
    bootstrap_dir = os.path.join(output_base_dir, f'bootstrap_{i}')
    os.makedirs(bootstrap_dir, exist_ok=True)

    # Process each sample
    for sample_id, sample_df in hrd_wgd_timing_samples.items():
        merged_df = []

        for group_label in ['Early', 'Late', 'NA']:
            group_df = sample_df[sample_df['Classification'] == group_label].copy()

            if len(group_df) == 0:
                continue  # Skip empty groups

            # Type probabilities
            type_probs = (
                group_df['SBS96']
                .value_counts(normalize=True)
                .reindex(mutation_types, fill_value=0.0)
            )

            # Multinomial sampling
            sample_size = len(group_df)
            sampled_counts = np.random.multinomial(sample_size, type_probs.values)
            sampled_types = np.repeat(type_probs.index.values, sampled_counts)
            sampled_types_series = pd.Series(sampled_types, name='SBS96')

            # Create counts DataFrame
            sampled_counts_df = pd.DataFrame(
                {sample_id: sampled_types_series.value_counts(normalize=False)
                 .reindex(mutation_types, fill_value=0.0)}
            )
            sampled_counts_df.index.name = 'Type'

            # Fit signatures
            sampled_exp, _ = musical.refit.refit(sampled_counts_df, W, method='likelihood_bidirectional', thresh=0.001)
            sampled_exp_norm = sampled_exp / sampled_exp.sum()
            sampled_exp_norm = sampled_exp_norm.iloc[:, 0]
            prob_matrix = W.mul(sampled_exp_norm, axis=1)

            prob_sum = prob_matrix.sum(axis=1)
            prob_df = prob_matrix.div(prob_sum, axis=0).fillna(0)
            prob_df = prob_df.rename(columns=lambda c: f'prob_{c}_boot').reset_index()

            # Annotate and merge with original
            group_df['Type'] = group_df['SBS96']
            group_df = group_df.merge(prob_df, on='Type', how='left')
            merged_df.append(group_df)

        # Merge all groups into one DataFrame for this sample
        if merged_df:
            final_df = pd.concat(merged_df, ignore_index=True)
            final_path = os.path.join(bootstrap_dir, f"{sample_id}.csv")
            final_df.to_csv(final_path, index=False)


Bootstrapping:  26%|██▌       | 52/200 [06:01<17:10,  6.96s/it]


KeyboardInterrupt: 

In [ ]:
def run_bootstrap_exposure_sampling(samples_dict, n_bootstraps, output_dir):
    """
    Perform bootstrap resampling of SBS96 mutations per sample and time classification
    ('Early', 'Late', 'NA') to estimate signature exposures using the COSMIC v3.2 catalog.

    For each bootstrap iteration:
    - For each sample and time group, mutations are resampled using multinomial sampling
      based on observed SBS96 mutation type frequencies.
    - Signature exposures are inferred via likelihood-based refitting using `musical`.
    - Posterior mutation probabilities per signature are calculated and merged with
      the original group data.
    - Annotated sample data are saved to per-sample CSV files.
    - For each group label, combined exposure vectors across all samples are saved as CSV.

    Parameters:
    ----------
    samples_dict : dict
        Dictionary mapping sample IDs to DataFrames. Each DataFrame must contain:
        - 'SBS96': mutation types (str)
        - 'Classification': one of {'Early', 'Late', 'NA'}

    n_bootstraps : int
        Number of bootstrap replicates to perform.

    output_dir : str
        Path to directory where bootstrap results will be saved. Each bootstrap replicate
        will be saved in a subdirectory named 'bootstrap_<i>'.

    Output:
    -------
    - For each bootstrap replicate:
        - <output_dir>/bootstrap_<i>/<sample_id>.csv:
            Sample-level annotated mutations with posterior signature probabilities.
        - <output_dir>/bootstrap_<i>/exposures_<label>.csv:
            Combined exposure matrix for each group label.
    """

    catalog = musical.load_catalog('COSMIC_v3p2_SBS_WGS')
    catalog.restrict_catalog(tumor_type='Breast.AdenoCA', is_MMRD=False, is_PPD=False)
    mutation_types = catalog.W.index.tolist()
    W = catalog.W.reindex(mutation_types)
    
    os.makedirs(output_dir, exist_ok=True)
    
    for i in tqdm(range(1, n_bootstraps + 1), desc="Bootstrapping"):
        boot_dir = os.path.join(output_dir, f'bootstrap_{i}')
        os.makedirs(boot_dir, exist_ok=True)

        group_exposure_dict = {'Early': [], 'Late': [], 'NA': []}
        
        for sample_id, df in samples_dict.items():
            merged = []

            for label in ['Early', 'Late', 'NA']:
                group = df[df['Classification'] == label].copy()
                if group.empty:
                    continue
                
                type_probs = group['SBS96'].value_counts(normalize=True).reindex(mutation_types, fill_value=0.0)
                sampled = np.random.multinomial(len(group), type_probs.values)
                sampled_types = np.repeat(type_probs.index.values, sampled)
                sampled_df = pd.DataFrame({sample_id: pd.Series(sampled_types).value_counts()
                                           .reindex(mutation_types, fill_value=0)}, index=mutation_types)
                sampled_df.index.name = 'Type'

                exposures, _ = musical.refit.refit(sampled_df, W, method='likelihood_bidirectional', thresh=0.001)
                exposures_norm = (exposures / exposures.sum()).iloc[:, 0]
                exposures_norm.name = sample_id
                exposures.name = sample_id
                group_exposure_dict[label].append(exposures)

                W_prob = W.mul(exposures_norm, axis=1)
                prob_df = W_prob.div(W_prob.sum(axis=1), axis=0).fillna(0).rename(
                    columns=lambda c: f'prob_{c}_boot').reset_index()

                group['Type'] = group['SBS96']
                group = group.merge(prob_df, on='Type', how='left')
                merged.append(group)

            if merged:
                pd.concat(merged, ignore_index=True).to_csv(os.path.join(boot_dir, f"{sample_id}.csv"), index=False)

        for label, exposure_list in group_exposure_dict.items():
            if exposure_list:
                df_exposures = pd.concat(exposure_list, axis=1).T
                df_exposures.to_csv(os.path.join(boot_dir, f"exposures_{label}.csv"))

output_dir = '/Users/michail/HMS Dropbox/Michail Andreopoulos/HRDTimer/temp/PCAWG_Breast_boot_only_p_change'
n_bootstraps = 200

HRDTimerUtils.generate_bootstraps(
    samples_dict=hrd_wgd_timing_samples,
    n_bootstraps=n_bootstraps,
    output_dir=output_dir
)

Bootstrapping:   0%|          | 0/200 [00:00<?, ?it/s]

Bootstrapping:  27%|██▋       | 54/200 [06:01<16:30,  6.78s/it]